# Section G - Web Scraping

Feedback: https://forms.gle/Le3RAsMEcYqEyswEA


Scraping modules
Javascript or no javascript
Navigating a page
Interaction - clicking elements or entering text data


In [17]:
from bs4 import BeautifulSoup
from urllib.request import urlopen

# url = "http://wikipedia.org"
url = "https://en.wikipedia.org/wiki/Python_(programming_language)"
page = urlopen(url)
html = page.read().decode('utf-8')
soup = BeautifulSoup(html, 'html.parser')
# print(soup.get_text())
print(dir(soup))
print(soup.title)
# print(soup.body.get_text())
# xpath = '//*[@id="mw-content-text"]/div[1]/p[2]'
xpath = '/html/body/div[2]/div/div[3]/main/div[3]/div[3]/div[1]/p[2]'
# print(soup.select_one(xpath).get_text())

['ASCII_SPACES', 'DEFAULT_BUILDER_FEATURES', 'DEFAULT_INTERESTING_STRING_TYPES', 'EMPTY_ELEMENT_EVENT', 'END_ELEMENT_EVENT', 'NO_PARSER_SPECIFIED_WARNING', 'ROOT_TAG_NAME', 'START_ELEMENT_EVENT', 'STRING_ELEMENT_EVENT', '__bool__', '__call__', '__class__', '__contains__', '__copy__', '__deepcopy__', '__delattr__', '__delitem__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattr__', '__getattribute__', '__getitem__', '__getstate__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__iter__', '__le__', '__len__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__setitem__', '__setstate__', '__sizeof__', '__str__', '__subclasshook__', '__unicode__', '__weakref__', '_all_strings', '_clone', '_decode_markup', '_event_stream', '_feed', '_find_all', '_find_one', '_format_tag', '_indent_string', '_is_xml', '_lastRecursiveChild', '_last_descendant', '_linkage_fixer', '_markup_is_url', '_markup_resembles_

In [24]:
# %pip install lxml

try:
    # Python 2
    from urllib2 import urlopen
except ImportError:
    from urllib.request import urlopen
from lxml import etree

response = urlopen(url)


In [25]:
htmlparser = etree.HTMLParser()
tree = etree.parse(response, htmlparser)
section = tree.xpath(xpath)
for item in section:
    print(item.text)

None



Following along with:
https://zperzan.github.io/projects/scrape-weather-underground/
Requires chromedriver from https://googlechromelabs.github.io/chrome-for-testing/
* set it in the chromedriver_path variable once installed.

In [ ]:

"""Module to scrape 5-min personal weather station data from Weather Underground.

Usage is:
>>> python scrape_wunderground.py   STATION    DATE

where station is a personal weather station (e.g., KCAJAMES3) and date is in the 
format YYYY-MM-DD. 

Alternatively, each function below can be imported and used in a separate python
script. Note that a working version of chromedriver must be installed and the absolute 
path to executable has to be updated below ("chromedriver_path").

Zach Perzan, 2021-07-28"""

import time
import sys

import numpy as np
import pandas as pd
from bs4 import BeautifulSoup as BS
from selenium import webdriver


# Set the absolute path to chromedriver
# chromedriver_path = '/home/_snap/bin/chromium.chromedriver'
# chromedriver_path = '/home/dan/opt/chrome-linux64'
chromedriver_path = '/home/dan/opt/geckodriver-v0.36.0-linux64/geckodriver'


def render_page(url):
   
    # driver = webdriver.Chrome(chromedriver_path)
    driver = webdriver.Firefox(chromedriver_path)
    driver.get(url)
    time.sleep(3) # Could potentially decrease the sleep time
    r = driver.page_source
    driver.quit()

    return r


def scrape_wunderground(station, date):
    
    # Render the url and open the page source as BS object
    # url = 'https://www.wunderground.com/dashboard/pws/%s/table/%s/%s/daily' % (station, date, date)
    url = f'https://www.wunderground.com/history/weekly/us/ca/sacramento/KSMF/date/2025-3-1'
    
    print("Working on url:", url)
    r = render_page(url)
    soup = BS(r, "html.parser",)

    # container = soup.find('lib-history-table')
    container = soup.find('lib-city-history-observation')
    
    # Check that lib-history-table is found
    if container is None:
        print('error finding the container...')
        raise ValueError("could not find lib-history-table in html source for %s" % url)
    
    print(container.contents)
    # Get the timestamps and data from two separate 'tbody' tags
    all_checks = container.find_all('tbody')
    time_check = all_checks[0]
    data_check = all_checks[1]
    print(time_check)
    print(data_check)

    # Iterate through 'tr' tags and get the timestamps
    hours = []
    for i in time_check.find_all('tr'):
        trial = i.get_text()
        hours.append(trial)

    # For data, locate both value and no-value ("--") classes
    # classes = ['wu-value wu-value-to', 'wu-unit-no-value ng-star-inserted']
    classes = ['observation-table', 'ng-star-inserted']

    # Iterate through span tags and get data
    data = []
    for i in data_check.find_all('span', class_=classes):
        trial = i.get_text()
        data.append(trial)

    columns = ['Time', 'Temperature', 'Dew Point', 'Humidity', 'Wind Speed', 
               'Wind Gust', 'Pressure', 'Precip.', 'Condition']

    # Convert NaN values (stings of '--') to np.nan
    data_nan = [np.nan if x == '--' else x for x in data]

    # Convert list of data to an array
    data_array = np.array(data_nan, dtype=float)
    data_array = data_array.reshape(-1, len(columns))

    # Prepend date to HH:MM strings
    timestamps = ['%s %s' % (date, t) for t in hours]

    # Convert to dataframe
    df = pd.DataFrame(index=timestamps, data=data_array, columns=columns)
    df.index = pd.to_datetime(df.index)
    
    return df


def scrape_multiattempt(station, date, attempts=4, wait_time=5.0):
    """Try to scrape data from Weather Underground. If there is an error on the 
    first attempt, try again.
    
    Parameters
    ----------
        station : str
            The personal weather station ID
        date : str
            The date for which to acquire data, formatted as 'YYYY-MM-DD'
        attempts : int, default 4
            Maximum number of times to try accessing before failuer
        wait_time : float, default 5.0
            Amount of time to wait in between attempts
            
    Returns
    -------
        df : dataframe or None
            A dataframe of weather observations, with index as pd.DateTimeIndex 
            and columns as the observed data
    """
    
    # Try to download data limited number of attempts
    for n in range(attempts):
        try:
            df = scrape_wunderground(station, date)
            print('success')
        except:
            print('exception')
            # if unsuccessful, pause and retry
            if attempts > 1:
                time.sleep(wait_time)
        else: 
            # if successful, then break
            break
    # If all attempts failed, return empty df
    else:
        df = pd.DataFrame()
        
    return df

In [59]:
# data = scrape_multiattempt('KSMF', '2025-03-01', 1)
# data = scrape_multiattempt('KCAJAMES3', '2025-03-01')
data = scrape_wunderground('KSMF', '2025-03-01')

Working on url: https://www.wunderground.com/history/weekly/us/ca/sacramento/KSMF/date/2025-3-1


AttributeError: 'str' object has no attribute 'capabilities'

In [33]:
print(data)

Empty DataFrame
Columns: []
Index: []


In [63]:
import time
import sys

import numpy as np
import pandas as pd
from bs4 import BeautifulSoup as BS
from selenium import webdriver
from selenium.webdriver.chrome.service import Service


# Set the absolute path to chromedriver
# chromedriver_path = '/home/_snap/bin/chromium.chromedriver'
# chromedriver_path = '/home/dan/opt/chrome-linux64/chrome'
chromedriver_path = '/home/dan/opt/chromedriver-linux64/chromedriver'
# chromedriver_path = '/home/dan/opt/geckodriver-v0.36.0-linux64/geckodriver'

url = f'https://www.wunderground.com/history/weekly/us/ca/sacramento/KSMF/date/2025-3-1'

# driver = webdriver.Chrome(chromedriver_path)
# driver = webdriver.Firefox(chromedriver_path)
service = Service(chromedriver_path)
driver = webdriver.Chrome(service=service)

driver.get(url)
time.sleep(3) # Could potentially decrease the sleep time
r = driver.page_source
driver.quit()



In [73]:
soup = BS(r, "html.parser",)

# container = soup.find('lib-history-table')
container = soup.find('lib-city-history-observation')
print(container.contents)

# column names are in thead
columns = ['Time', 'Temperature', 'Dew Point', 'Humidity', 'Wind Speed', 
               'Wind Gust', 'Pressure', 'Precip.', 'Condition']
# confirm that our columns are correct:
column_headers = container.find_all('thead')
# break it up by column:
for header in column_headers:
    for item in header:
        print(item.get_text())

# data rows are in tbody


[<div _ngcontent-app-root-c196=""><div _ngcontent-app-root-c196="" class="observation-title">Daily Observations</div><div _ngcontent-app-root-c196="" class="observation-table ng-star-inserted"><table _ngcontent-app-root-c196="" aria-labelledby="History days" class="days ng-star-inserted"><thead _ngcontent-app-root-c196=""><tr _ngcontent-app-root-c196=""><td _ngcontent-app-root-c196="" class="ng-star-inserted">Time</td><td _ngcontent-app-root-c196="" class="ng-star-inserted">Temperature (°F)</td><td _ngcontent-app-root-c196="" class="ng-star-inserted">Dew Point (°F)</td><td _ngcontent-app-root-c196="" class="ng-star-inserted">Humidity (%)</td><td _ngcontent-app-root-c196="" class="ng-star-inserted">Wind Speed (mph)</td><td _ngcontent-app-root-c196="" class="ng-star-inserted">Pressure (in)</td><td _ngcontent-app-root-c196="" class="ng-star-inserted">Precipitation (in)</td><!-- --></tr></thead><tbody _ngcontent-app-root-c196=""><tr _ngcontent-app-root-c196=""><td _ngcontent-app-root-c196=